In [1]:
import pandas as pd
from ensemble import StdForestClassifier
from category_encoders.m_estimate import MEstimateEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,classification_report
from sklearn.ensemble import RandomForestClassifier
import shap
import numpy as np
import matplotlib.pyplot as plt
from imblearn.under_sampling import RandomUnderSampler

In [2]:
df = pd.read_csv('data/train_transaction.csv')

In [3]:
df = df.iloc[:, :-300]
#df = df.sample(frac=0.1)
undersample = RandomUnderSampler(sampling_strategy=0.5)

In [4]:
X = df.drop(columns=['TransactionID', 'isFraud', 'TransactionDT'])
y = df[['isFraud']]

In [5]:
X,y = undersample.fit_resample(X,y)

In [6]:
X = X.fillna(-1)

In [7]:
X = MEstimateEncoder().fit_transform(X,y)

is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead


In [8]:
X_tr, X_te, y_tr, y_te = train_test_split(X,y,stratify=y,test_size=0.1)

In [9]:
rf = RandomForestClassifier(n_estimators= 10)
rf = StdForestClassifier()

In [10]:
%%time
rf.fit(X_tr,y_tr)

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


CPU times: user 9.67 s, sys: 94.2 ms, total: 9.77 s
Wall time: 9.81 s


In [13]:
print(classification_report(rf.predict(X_te),y_te))

              precision    recall  f1-score   support

           0       0.96      0.90      0.93      4421
           1       0.78      0.90      0.84      1778

    accuracy                           0.90      6199
   macro avg       0.87      0.90      0.88      6199
weighted avg       0.91      0.90      0.90      6199



In [17]:
rf.predict_std(X_tr)[:,0]

array([0.32496154, 0.19595918, 0.17058722, ..., 0.09949874, 0.14      ,
       0.2712932 ])